In [86]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray


'''

import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory

In [126]:
##################################################
# Load  
##################################################
input_col = ['msno','transaction_date','early_expiration_days']
membership_loyalty = utils.read_multiple_csv('../../input/preprocessed_data/transactions_date_base') # 20,000,000

#membership_loyalty = membership_loyalty.head(n = 500)

#################################################
#Convert string to datetime format
#################################################
membership_loyalty['transaction_date']  = membership_loyalty.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(membership_loyalty)


100%|██████████| 4/4 [00:22<00:00,  5.57s/it]


reduce memory



100%|██████████| 8/8 [00:00<00:00, 1578.88it/s]

In [101]:
membership_loyalty.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22964064 entries, 0 to 5741015
Data columns (total 8 columns):
msno                                            object
transaction_date                                datetime64[ns]
membership_duration                             int16
is_membership_duration_equal_to_plan_days       int8
is_membership_duration_longer_than_plan_days    int8
days_longer_than_plan_days                      int16
is_early_expiration                             int8
early_expiration_days                           int8
dtypes: datetime64[ns](1), int16(2), int8(4), object(1)
memory usage: 700.8+ MB


In [102]:
membership_loyalty[input_col].head()

,msno,transaction_date,early_expiration_days
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,2016-09-09,0
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2015-11-21,0
2,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2016-10-23,0
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-16,0
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-12-15,0


In [129]:
membership_loyalty['early_expiration_days'] = membership_loyalty.early_expiration_days.apply(lambda x : x* -1 if x < 0 else x)

In [130]:
tbl = membership_loyalty.early_expiration_days.value_counts().reset_index()
tbl.columns = ['early_expiration_days', 'early_expiration_days_cnt']

In [131]:
tbl

,early_expiration_days,early_expiration_days_cnt
0,0,22810285
1,1,99841
2,2,42888
3,3,1038
4,31,597
5,30,554
6,6,354
7,4,349
8,61,322
9,32,319


# core

In [111]:
T = 0
folder = 'trainW-'+str(T)
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno']]



In [112]:
df = pd.merge(train, 
membership_loyalty[(membership_loyalty.transaction_date < datetime.strptime('2017-03-01', '%Y-%m-%d'))], 
on=['msno'], 
how='left')

In [114]:
df.shape

(14748281, 8)

In [117]:
#core
tbl = df.head(n = 1000).groupby('msno').early_expiration_days.mean().to_frame()
tbl.columns = ['early_expiration_days-mean']
tbl['early_expiration_days-min'] = df.groupby('msno').early_expiration_days.min()
tbl['early_expiration_days-max'] = df.groupby('msno').early_expiration_days.max()
tbl['early_expiration_days-median'] = df.groupby('msno').early_expiration_days.median()
tbl['early_expiration_days-std'] = df.groupby('msno').early_expiration_days.std()

In [118]:
tbl.reset_index(inplace = True)
tbl

,msno,early_expiration_days-mean,early_expiration_days-min,early_expiration_days-max,early_expiration_days-median,early_expiration_days-std
0,+3KltBa/1dUuXwOzDKksw11Nwdwf7/pXv47sDv4mInY=,0.000000,0,0,0.0,0.000000
1,+wisdxP166qHezQOKLPv4l4AfZClRF3ExWGD2rzYYCU=,0.000000,0,0,0.0,0.000000
2,/66xO0tgKHeHrtaULlc3sOtEysp2v2y+kjcaqXScvyo=,0.000000,0,0,0.0,0.000000
3,/L2095JD4M/BNLTCboucgyuT2i8JIiJ87bOIsxtFvBw=,0.000000,0,0,0.0,0.000000
4,07BqZinAC48xGnZjsnufxMnduRW0M3Tcs7hTK62ygTs=,0.000000,0,0,0.0,0.000000
5,1AzXWFlRO6EfMBzfBdk98sBVnjzY7U1G24mVFNdzGNQ=,0.000000,0,0,0.0,0.000000
6,2OaV7/7vyAA2FrCkV4qXZEZYg5IzVyFNkQnQorgQ440=,2.694444,0,97,0.0,16.166667
7,3s5Bn4gcF5NrGMmG62c9hLVitansQ4JtLpz58c7g8JU=,0.000000,0,0,0.0,0.000000
8,6OlZbbgM+0PjnShnk82ORhLuBaqn6m0bv+I/x9bpmHs=,0.000000,0,0,0.0,0.000000
9,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,7.655172,0,111,0.0,28.624759


# early_expiration_days is negative

In [17]:
membership_loyalty[membership_loyalty.early_expiration_days == -14]

,msno,membership_duration,is_membership_duration_equal_to_plan_days,is_membership_duration_longer_than_plan_days,days_longer_than_plan_days,is_early_expiration,early_expiration_days
652506,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,-242,0,0,0,1,-14
805305,0CAvDGWLGOlENFiOkSCZd2fzlAk5ptOFXd+ddfNwRqA=,-242,0,0,0,1,-14
943941,0bAK8hFztHoYGFNDktYu66rw4KWeOnvcnlLxsuXSXSw=,-242,0,0,0,1,-14
1842736,35XirzhOUaERiHV8xIDCEzQWMCZCIVWosM+w3IoT+N8=,-498,0,0,0,1,-14
4374077,A8dNrVvbFsST+ZIjumOPQhTtOdxGwoZrvY0C6j5O3Vw=,-498,0,0,0,1,-14
805475,GDjrSQADdF+R6wi5ziWKjuQ4kWy6YTH5nPTbi2yfyWU=,-498,0,0,0,1,-14
972569,GhURQZx4JXoVxPFYiIzkyaEjMbNZb+cyNp8Sjse6pGA=,-498,0,0,0,1,-14
2881017,M/0soEWDMQrfeKLFc6ItRFOVsnhH2K11pX3nZBAY950=,-242,0,0,0,1,-14
4665551,R+HwHTH1SUSb47spQEm6f7RQaN4d7MwCxaS4XAbCo8U=,-498,0,0,0,1,-14
5722892,TwXSMWlDEM+C5+vXkfmGqlktAOv8mJDy/sfsbTC8/WE=,-498,0,0,0,1,-14


In [42]:
##################################################
# Load transaction 
##################################################
input_col = ['msno','payment_plan_days','transaction_date', 'membership_expire_date',]
transactions = utils.read_multiple_csv('../../input/preprocessed_data/transactions',input_col)

100%|██████████| 4/4 [00:27<00:00,  6.98s/it]


In [65]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22964064 entries, 0 to 5741015
Data columns (total 4 columns):
msno                      object
transaction_date          object
payment_plan_days         int16
membership_expire_date    object
dtypes: int16(1), object(3)
memory usage: 744.6+ MB


In [66]:
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(transactions)
transactions = transactions[transactions.msno == '/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=']
##################################################
# Convert string to datetime format
##################################################
transactions['membership_expire_date']  = transactions.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
transactions['transaction_date']  = transactions.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))


reduce memory


100%|██████████| 4/4 [00:00<00:00, 3783.77it/s]
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [67]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 652489 to 652522
Data columns (total 4 columns):
msno                      34 non-null object
transaction_date          34 non-null datetime64[ns]
payment_plan_days         34 non-null int16
membership_expire_date    34 non-null datetime64[ns]
dtypes: datetime64[ns](2), int16(1), object(1)
memory usage: 1.1+ KB


In [68]:
# 幹為什麼會有負的i.days

In [69]:
transactions['early_expiration_days'] = [-i.days if i.days < 0 else 0 for i in (transactions.membership_expire_date - transactions.transaction_date)]
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 652489 to 652522
Data columns (total 5 columns):
msno                      34 non-null object
transaction_date          34 non-null datetime64[ns]
payment_plan_days         34 non-null int16
membership_expire_date    34 non-null datetime64[ns]
early_expiration_days     34 non-null int64
dtypes: datetime64[ns](2), int16(1), int64(1), object(1)
memory usage: 2.6+ KB


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [81]:
# 幹不存在負的
output_col = ['msno','transaction_date','early_expiration_days']
transactions = transactions[output_col]
transactions

,msno,transaction_date,early_expiration_days
652489,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-01-31,0
652490,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-02-28,0
652491,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-03-31,0
652492,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-04-26,0
652493,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-04-30,0
652494,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-05-25,0
652495,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-05-31,0
652496,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-06-25,0
652497,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-06-30,0
652498,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-07-25,0


In [78]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 652489 to 652522
Data columns (total 3 columns):
msno                     34 non-null object
transaction_date         34 non-null datetime64[ns]
early_expiration_days    34 non-null int64
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.3+ KB


In [61]:
transactions.head()

,msno,transaction_date,payment_plan_days,membership_expire_date
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,2016-09-09,7,2016-09-14
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2015-11-21,410,2017-01-04
2,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2016-10-23,395,2018-02-06
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-16,30,2016-12-15
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-12-15,30,2017-01-15


# why membership_loyalty 會差一個負號

In [119]:
membership_loyalty = utils.read_multiple_csv('../../input/preprocessed_data/transactions_date_base') # 20,000,000


100%|██████████| 4/4 [00:23<00:00,  5.80s/it]


In [83]:
membership_loyalty.info()
membership_loyalty['early_expiration_days'] = membership_loyalty.early_expiration_days.apply(lambda x : x* -1 if x < 0 else x)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22964064 entries, 0 to 5741015
Data columns (total 8 columns):
msno                                            object
transaction_date                                object
membership_duration                             int64
is_membership_duration_equal_to_plan_days       int64
is_membership_duration_longer_than_plan_days    int64
days_longer_than_plan_days                      int64
is_early_expiration                             int64
early_expiration_days                           int64
dtypes: int64(6), object(2)
memory usage: 1.5+ GB


In [121]:
output_col = ['msno','transaction_date','early_expiration_days']
membership_loyalty[membership_loyalty.msno == '/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk='][output_col]

,msno,transaction_date,early_expiration_days
652489,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-01-31,0
652490,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-02-28,0
652491,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-03-31,0
652492,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-04-26,0
652493,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-04-30,0
652494,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-05-25,0
652495,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-05-31,0
652496,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-06-25,0
652497,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-06-30,0
652498,/nPDKKeCsp3ifkeonrSdvdJrtSadkJRnwVTrV8RV/rk=,2015-07-25,0


In [85]:
membership_loyalty[membership_loyalty.early_expiration_days < 0]

,msno,transaction_date,membership_duration,is_membership_duration_equal_to_plan_days,is_membership_duration_longer_than_plan_days,days_longer_than_plan_days,is_early_expiration,early_expiration_days
